Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [12]:
''' First - adding it the stochastic logistic regression done on 2-fullyconnected '''
batch_size = 128

graph = tf.Graph()
with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    beta_val = tf.placeholder(tf.float32)  
    weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    loss = loss + beta_val * tf.nn.l2_loss(weights)
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [16]:
num_steps = 3001
beta = 0.01 # constant - added 0.3% accuracy
beta = 0.1 # constant - added 0.7% accuracy

def runSession(beta):
    with tf.Session(graph=graph) as session:
      tf.global_variables_initializer().run()
      print("Initialized")
      for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, beta_val : beta}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
            print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))
beta_vals = np.logspace(-4, -2, 20)
for b in beta_vals:
    print('-----------')
    print('beta:',b)
    runSession(b)

-----------
beta: 0.0001
Initialized
Minibatch loss at step 0: 15.981618
Minibatch accuracy: 10.2%
Validation accuracy: 9.8%
Test accuracy: 9.4%
Minibatch loss at step 500: 1.503785
Minibatch accuracy: 77.3%
Validation accuracy: 75.8%
Test accuracy: 83.6%
Minibatch loss at step 1000: 1.714602
Minibatch accuracy: 75.8%
Validation accuracy: 76.1%
Test accuracy: 84.1%
Minibatch loss at step 1500: 0.843820
Minibatch accuracy: 85.2%
Validation accuracy: 77.0%
Test accuracy: 85.1%
Minibatch loss at step 2000: 1.036397
Minibatch accuracy: 82.8%
Validation accuracy: 77.6%
Test accuracy: 85.4%
Minibatch loss at step 2500: 1.181303
Minibatch accuracy: 77.3%
Validation accuracy: 78.6%
Test accuracy: 86.6%
Minibatch loss at step 3000: 1.089244
Minibatch accuracy: 77.3%
Validation accuracy: 79.2%
Test accuracy: 86.6%
-----------
beta: 0.00012742749857031334
Initialized
Minibatch loss at step 0: 17.606117
Minibatch accuracy: 8.6%
Validation accuracy: 9.3%
Test accuracy: 9.2%
Minibatch loss at step 5

Initialized
Minibatch loss at step 0: 20.230251
Minibatch accuracy: 14.1%
Validation accuracy: 15.8%
Test accuracy: 16.3%
Minibatch loss at step 500: 2.567128
Minibatch accuracy: 76.6%
Validation accuracy: 76.1%
Test accuracy: 83.9%
Minibatch loss at step 1000: 1.859608
Minibatch accuracy: 79.7%
Validation accuracy: 79.0%
Test accuracy: 86.5%
Minibatch loss at step 1500: 0.848027
Minibatch accuracy: 86.7%
Validation accuracy: 80.0%
Test accuracy: 87.5%
Minibatch loss at step 2000: 0.804425
Minibatch accuracy: 86.7%
Validation accuracy: 80.8%
Test accuracy: 87.8%
Minibatch loss at step 2500: 0.836356
Minibatch accuracy: 78.1%
Validation accuracy: 81.6%
Test accuracy: 88.8%
Minibatch loss at step 3000: 0.733899
Minibatch accuracy: 82.8%
Validation accuracy: 81.8%
Test accuracy: 88.8%
-----------
beta: 0.0014384498882876629
Initialized
Minibatch loss at step 0: 23.201576
Minibatch accuracy: 8.6%
Validation accuracy: 14.6%
Test accuracy: 15.3%
Minibatch loss at step 500: 2.624696
Minibatch

In [6]:
''' Adding the l2-loss func to the nn with 1 hidden layer from 2-fullyconnected '''
num_steps = 3001
batch_size = 128
hidden_nodes = 1024
image_size = 28
num_labels = 10

graph = tf.Graph()
with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    beta_val = tf.placeholder(tf.float32)
    # Variables.
    #   first we have matrix that is from the size of the input (image_size^2) to the size of the hidden nodes
    w_1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_nodes]))
    b_1 = tf.Variable(tf.zeros([hidden_nodes]))
    #   second we have matrix that is from the size of the hidden nodes to the size of the inputs
    w_2 = tf.Variable(tf.truncated_normal([hidden_nodes, num_labels]))
    b_2 = tf.Variable(tf.zeros([num_labels]))

    def ForwardProp(dataset):
        z1 = tf.matmul(dataset, w_1) + b_1 # first variables
        a1 = tf.nn.relu(z1) # activation function ReLU
        z2 = tf.matmul(a1, w_2) + b_2
        return z2

    # Training computation.
    logits = ForwardProp(tf_train_dataset)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) \
        + beta_val * ( tf.nn.l2_loss(w_1) + tf.nn.l2_loss(w_1) + tf.nn.l2_loss(b_1) + tf.nn.l2_loss(b_2))

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax( ForwardProp(tf_valid_dataset) )
    test_prediction = tf.nn.softmax( ForwardProp(tf_test_dataset) )

In [8]:
num_steps = 3001

def runSession(beta):
    with tf.Session(graph=graph) as session:
      tf.global_variables_initializer().run()
      print("Initialized")
      for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, beta_val : beta}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
          print("Minibatch loss at step %d: %f" % (step, l))
          print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
          print("Validation accuracy: %.1f%%" % accuracy(
            valid_prediction.eval(), valid_labels))
      print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))
beta_vals = np.logspace(-4, -2, 5)
for b in beta_vals:
    print('-----------')
    print('beta:',b)
    runSession(b)

-----------
beta: 0.0001
Initialized
Minibatch loss at step 0: 349.900818
Minibatch accuracy: 14.8%
Validation accuracy: 30.7%
Minibatch loss at step 500: 76.754547
Minibatch accuracy: 78.9%
Validation accuracy: 80.7%
Minibatch loss at step 1000: 59.571159
Minibatch accuracy: 79.7%
Validation accuracy: 81.7%
Minibatch loss at step 1500: 48.234573
Minibatch accuracy: 87.5%
Validation accuracy: 80.8%
Minibatch loss at step 2000: 43.094162
Minibatch accuracy: 85.2%
Validation accuracy: 81.5%
Minibatch loss at step 2500: 38.955467
Minibatch accuracy: 80.5%
Validation accuracy: 82.7%
Minibatch loss at step 3000: 34.016869
Minibatch accuracy: 84.4%
Validation accuracy: 82.9%
Test accuracy: 90.5%
-----------
beta: 0.00031622776601683794
Initialized
Minibatch loss at step 0: 587.710571
Minibatch accuracy: 14.1%
Validation accuracy: 25.4%
Minibatch loss at step 500: 155.180008
Minibatch accuracy: 79.7%
Validation accuracy: 80.2%
Minibatch loss at step 1000: 111.188232
Minibatch accuracy: 77.3%


In [ ]:
# Note that in 2-fullyconnected those was the results:
#
# SGD LR - Test accuracy: 86.1%
# NN with 1 hidden layer - Test accuracy: 90.0%
#
# Here for NN with beta of 0.001 Test accuracy: 92.8%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [26]:
num_steps = 3001
batch_size = 128
larger_batch_size = 5 * batch_size
hidden_nodes = 1024
image_size = 28
num_labels = 10

graph = tf.Graph()
with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    beta_val = tf.placeholder(tf.float32)
    # Variables.
    #   first we have matrix that is from the size of the input (image_size^2) to the size of the hidden nodes
    w_1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_nodes]))
    b_1 = tf.Variable(tf.zeros([hidden_nodes]))
    #   second we have matrix that is from the size of the hidden nodes to the size of the inputs
    w_2 = tf.Variable(tf.truncated_normal([hidden_nodes, num_labels]))
    b_2 = tf.Variable(tf.zeros([num_labels]))

    def ForwardProp(dataset):
        z1 = tf.matmul(dataset, w_1) + b_1 # first variables
        a1 = tf.nn.relu(z1) # activation function ReLU
        z2 = tf.matmul(a1, w_2) + b_2
        return z2

    # Training computation.
    logits = ForwardProp(tf_train_dataset)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) \
        + beta_val * ( tf.nn.l2_loss(w_1) + tf.nn.l2_loss(w_1) + tf.nn.l2_loss(b_1) + tf.nn.l2_loss(b_2))

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax( ForwardProp(tf_valid_dataset) )
    test_prediction = tf.nn.softmax( ForwardProp(tf_test_dataset) )

In [27]:
num_steps = 3001
small_train_labels = train_labels[:larger_batch_size, :]
small_train_dataset = train_dataset[:larger_batch_size, :]

def runSession(beta):
    with tf.Session(graph=graph) as session:
      tf.global_variables_initializer().run()
      print("Initialized")
      for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (small_train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = small_train_dataset[offset:(offset + batch_size), :]
        batch_labels = small_train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, beta_val : beta}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
          print("Minibatch loss at step %d: %f" % (step, l))
          print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
          print("Validation accuracy: %.1f%%" % accuracy(
            valid_prediction.eval(), valid_labels))
      print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))
beta_vals = [0.001] #np.logspace(-4, -2, 5)
for b in beta_vals:
    print('-----------')
    print('beta:',b)
    runSession(b)

-----------
beta: 0.001
Initialized
Minibatch loss at step 0: 979.323547
Minibatch accuracy: 7.0%
Validation accuracy: 25.5%
Minibatch loss at step 500: 227.557083
Minibatch accuracy: 100.0%
Validation accuracy: 76.3%
Minibatch loss at step 1000: 83.679993
Minibatch accuracy: 100.0%
Validation accuracy: 76.3%
Minibatch loss at step 1500: 30.774004
Minibatch accuracy: 100.0%
Validation accuracy: 76.3%
Minibatch loss at step 2000: 11.318583
Minibatch accuracy: 100.0%
Validation accuracy: 76.2%
Minibatch loss at step 2500: 4.163763
Minibatch accuracy: 100.0%
Validation accuracy: 76.4%
Minibatch loss at step 3000: 1.532303
Minibatch accuracy: 100.0%
Validation accuracy: 76.5%
Test accuracy: 84.4%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [33]:
num_steps = 3001
batch_size = 128
larger_batch_size = 5 * batch_size
hidden_nodes = 1024
image_size = 28
num_labels = 10

graph = tf.Graph()
with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    beta_val = tf.placeholder(tf.float32)
    # Variables.
    #   first we have matrix that is from the size of the input (image_size^2) to the size of the hidden nodes
    w_1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_nodes]))
    b_1 = tf.Variable(tf.zeros([hidden_nodes]))
    #   second we have matrix that is from the size of the hidden nodes to the size of the inputs
    w_2 = tf.Variable(tf.truncated_normal([hidden_nodes, num_labels]))
    b_2 = tf.Variable(tf.zeros([num_labels]))

    def ForwardProp(dataset, with_dropout=False):
        z1 = tf.matmul(dataset, w_1) + b_1 # first variables
        if (with_dropout):
            z1 = tf.nn.dropout(z1, 0.7)
        a1 = tf.nn.relu(z1) # activation function ReLU
        z2 = tf.matmul(a1, w_2) + b_2
        return z2

    # Training computation.
    logits = ForwardProp(tf_train_dataset, with_dropout=True)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) \
        + beta_val * ( tf.nn.l2_loss(w_1) + tf.nn.l2_loss(w_1) + tf.nn.l2_loss(b_1) + tf.nn.l2_loss(b_2))

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax( ForwardProp(tf_valid_dataset) )
    test_prediction = tf.nn.softmax( ForwardProp(tf_test_dataset) )
    

In [35]:
num_steps = 3001
small_train_labels = train_labels[:larger_batch_size, :]
small_train_dataset = train_dataset[:larger_batch_size, :]

def runSession(beta):
    with tf.Session(graph=graph) as session:
      tf.global_variables_initializer().run()
      print("Initialized")
      for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, beta_val : beta}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
          print("Minibatch loss at step %d: %f" % (step, l))
          print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
          print("Validation accuracy: %.1f%%" % accuracy(
            valid_prediction.eval(), valid_labels))
      print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))
beta_vals = np.logspace(-4, -2, 5)
for b in beta_vals:
    print('-----------')
    print('beta:',b)
    runSession(b)

-----------
beta: 0.0001
Initialized
Minibatch loss at step 0: 460.147644
Minibatch accuracy: 7.0%
Validation accuracy: 30.5%
Minibatch loss at step 500: 75.802155
Minibatch accuracy: 79.7%
Validation accuracy: 79.7%
Minibatch loss at step 1000: 63.497917
Minibatch accuracy: 74.2%
Validation accuracy: 80.6%
Minibatch loss at step 1500: 50.859657
Minibatch accuracy: 85.2%
Validation accuracy: 81.3%
Minibatch loss at step 2000: 43.677429
Minibatch accuracy: 81.2%
Validation accuracy: 81.5%
Minibatch loss at step 2500: 38.169109
Minibatch accuracy: 82.0%
Validation accuracy: 82.2%
Minibatch loss at step 3000: 34.211830
Minibatch accuracy: 82.8%
Validation accuracy: 82.0%
Test accuracy: 89.7%
-----------
beta: 0.00031622776601683794
Initialized
Minibatch loss at step 0: 607.953186
Minibatch accuracy: 5.5%
Validation accuracy: 21.9%
Minibatch loss at step 500: 168.043121
Minibatch accuracy: 79.7%
Validation accuracy: 79.5%
Minibatch loss at step 1000: 112.675270
Minibatch accuracy: 75.8%
Va

KeyboardInterrupt: 

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [47]:
''' First - Trying this learning rate decay '''
num_steps = 3001
batch_size = 128
larger_batch_size = 5 * batch_size
hidden_nodes = 1024
image_size = 28
num_labels = 10

graph = tf.Graph()
with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    beta_val = tf.placeholder(tf.float32)
    # Variables.
    #   first we have matrix that is from the size of the input (image_size^2) to the size of the hidden nodes
    w_1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_nodes]))
    b_1 = tf.Variable(tf.zeros([hidden_nodes]))
    #   second we have matrix that is from the size of the hidden nodes to the size of the inputs
    w_2 = tf.Variable(tf.truncated_normal([hidden_nodes, num_labels]))
    b_2 = tf.Variable(tf.zeros([num_labels]))

    def ForwardProp(dataset, with_dropout=False):
        z1 = tf.matmul(dataset, w_1) + b_1 # first variables
        if (with_dropout):
            z1 = tf.nn.dropout(z1, 0.6)
        a1 = tf.nn.relu(z1) # activation function ReLU
        z2 = tf.matmul(a1, w_2) + b_2
        return z2

    # Training computation.
    logits = ForwardProp(tf_train_dataset, with_dropout=True)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) \
        + beta_val * ( tf.nn.l2_loss(w_1) + tf.nn.l2_loss(w_2) + tf.nn.l2_loss(b_1) + tf.nn.l2_loss(b_2))

    # Optimizer.
    global_step = tf.Variable(0)  # count the number of steps taken.
    '''
    exponential_decay(
    learning_rate, -> A scalar float32 or float64 Tensor or a Python number. The initial learning rate.
    global_step, -> Global step to use for the decay computation. Must not be negative.
    decay_steps, -> 100
    decay_rate, -> 
    staircase=False,
    name=None)
    
    returns decayed_learning_rate = learning_rate *
                        decay_rate ^ (global_step / decay_steps)
    '''
    learning_rate = tf.train.exponential_decay(0.5, global_step, 1000, 0.5, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax( ForwardProp(tf_valid_dataset) )
    test_prediction = tf.nn.softmax( ForwardProp(tf_test_dataset) )
    

In [48]:
num_steps = 3001
small_train_labels = train_labels[:larger_batch_size, :]
small_train_dataset = train_dataset[:larger_batch_size, :]

def runSession(beta):
    with tf.Session(graph=graph) as session:
      tf.global_variables_initializer().run()
      print("Initialized")
      for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, beta_val : beta}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
          print("Minibatch loss at step %d: %f" % (step, l))
          print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
          print("Validation accuracy: %.1f%%" % accuracy(
            valid_prediction.eval(), valid_labels))
      print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))
beta_vals = [0.001] #np.logspace(-4, -2, 3)
for b in beta_vals:
    print('-----------')
    print('beta:',b)
    runSession(b)

-----------
beta: 0.001
Initialized
Minibatch loss at step 0: 704.741821
Minibatch accuracy: 7.8%
Validation accuracy: 27.8%
Minibatch loss at step 500: 206.052902
Minibatch accuracy: 78.1%
Validation accuracy: 80.4%
Minibatch loss at step 1000: 118.261208
Minibatch accuracy: 75.8%
Validation accuracy: 80.5%
Minibatch loss at step 1500: 89.941528
Minibatch accuracy: 83.6%
Validation accuracy: 81.6%
Minibatch loss at step 2000: 69.249283
Minibatch accuracy: 85.2%
Validation accuracy: 84.0%
Minibatch loss at step 2500: 61.058102
Minibatch accuracy: 83.6%
Validation accuracy: 84.5%
Minibatch loss at step 3000: 53.700108
Minibatch accuracy: 82.8%
Validation accuracy: 84.7%
Test accuracy: 91.7%


In [9]:
''' Multilayer '''
num_steps = 6001
batch_size = 128
hn_1 = 2048
hn_2 = 1024
hn_3 = 512
image_size = 28
num_labels = 10

graph = tf.Graph()
with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    beta_val = tf.placeholder(tf.float32)
    
    # Variables.
    w_1 = tf.Variable(
        tf.truncated_normal( [image_size * image_size, hn_1], stddev=np.sqrt(2.0 / (image_size * image_size) )))
    b_1 = tf.Variable(tf.zeros([hn_1]))
    w_2 = tf.Variable(tf.truncated_normal([hn_1, hn_2], stddev=np.sqrt(2. / hn_1)))
    b_2 = tf.Variable(tf.zeros([hn_2]))
    w_3 = tf.Variable(tf.truncated_normal([hn_2, hn_3], stddev=np.sqrt(2. / hn_2)))
    b_3 = tf.Variable(tf.zeros([hn_3]))
    w_4 = tf.Variable(tf.truncated_normal([hn_3, num_labels], stddev=np.sqrt(2. / hn_3)))
    b_4 = tf.Variable(tf.zeros([num_labels]))
    lr = tf.Variable(0.5)
    
    def ForwardProp(dataset):
        z1 = tf.nn.relu(tf.matmul(dataset, w_1) + b_1)
        z2 = tf.nn.relu(tf.matmul(z1, w_2) + b_2)
        z3 = tf.nn.relu(tf.matmul(z2, w_3) + b_3)
        z4 = tf.matmul(z3, w_4) + b_4
        return z4

    ''' Loss - log the loss and see its behavior and change learning rate (try linear) '''
    ''' batch size - try to play with. '''
    # Training computation.
    logits = ForwardProp(tf_train_dataset)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) \
        + beta_val * ( tf.nn.l2_loss(w_1) + tf.nn.l2_loss(w_2) + tf.nn.l2_loss(w_3) + tf.nn.l2_loss(w_4)) \
        + beta_val * ( tf.nn.l2_loss(b_1) + tf.nn.l2_loss(b_2) + tf.nn.l2_loss(b_3) + tf.nn.l2_loss(b_4))

    # Optimizer.
    global_step = tf.Variable(0)  # count the number of steps taken.
    
    learning_rate = tf.train.exponential_decay(lr, global_step, 1000, 0.5, staircase=True)
    if (global_step % 100 == 0):
        lr.assign(lr - 0.0001)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax( ForwardProp(tf_valid_dataset) )
    test_prediction = tf.nn.softmax( ForwardProp(tf_test_dataset) )

In [10]:
def runSession(beta):
    with tf.Session(graph=graph) as session:
      tf.global_variables_initializer().run()
      print("Initialized")
      for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, beta_val : beta}
        # the return value is the same as the first argument (named "parameters")]
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
          print("Minibatch loss at step %d: %f" % (step, l))
          print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
          print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
      print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))
beta_vals = np.random.choice(np.logspace(-4,-2,20), 1) # [0.001] #np.logspace(-4, -2, 3)
for b in beta_vals:
    print('-----------')
    print('beta:',b)
    runSession(b)

-----------
beta: 0.0003359818286283781
Initialized
Minibatch loss at step 0: 3.280884
Minibatch accuracy: 14.1%
Validation accuracy: 25.9%
Minibatch loss at step 500: 1.159702
Minibatch accuracy: 89.1%
Validation accuracy: 86.3%
Minibatch loss at step 1000: 1.183214
Minibatch accuracy: 85.2%
Validation accuracy: 87.4%
Minibatch loss at step 1500: 0.886158
Minibatch accuracy: 93.0%
Validation accuracy: 88.8%
Minibatch loss at step 2000: 0.830722
Minibatch accuracy: 94.5%
Validation accuracy: 89.2%
Minibatch loss at step 2500: 0.832374
Minibatch accuracy: 90.6%
Validation accuracy: 89.8%
Minibatch loss at step 3000: 0.866664
Minibatch accuracy: 89.8%
Validation accuracy: 90.0%
Minibatch loss at step 3500: 0.869088
Minibatch accuracy: 90.6%
Validation accuracy: 90.3%
Minibatch loss at step 4000: 0.771966
Minibatch accuracy: 93.0%
Validation accuracy: 90.3%
Minibatch loss at step 4500: 0.756263
Minibatch accuracy: 93.0%
Validation accuracy: 90.6%
Minibatch loss at step 5000: 0.816977
Mini

# The above one reached 96.0%

This one uses polynomial learning rate with power of 1 (that is - it is linear).
For start alpha 0.1 -> 95.5%
For start alpha 0.5 -> 96.2%
For start alpha 0.9 & 12000 steps -> 96.8%

In [19]:
''' Multilayer '''
num_steps = 12001
batch_size = 128
hn_1 = 2048
hn_2 = 1024
hn_3 = 512
image_size = 28
num_labels = 10

graph = tf.Graph()
with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    beta_val = tf.placeholder(tf.float32)
    
    # Variables.
    w_1 = tf.Variable(
        tf.truncated_normal( [image_size * image_size, hn_1], stddev=np.sqrt(2.0 / (image_size * image_size) )))
    b_1 = tf.Variable(tf.zeros([hn_1]))
    w_2 = tf.Variable(tf.truncated_normal([hn_1, hn_2], stddev=np.sqrt(2. / hn_1)))
    b_2 = tf.Variable(tf.zeros([hn_2]))
    w_3 = tf.Variable(tf.truncated_normal([hn_2, hn_3], stddev=np.sqrt(2. / hn_2)))
    b_3 = tf.Variable(tf.zeros([hn_3]))
    w_4 = tf.Variable(tf.truncated_normal([hn_3, num_labels], stddev=np.sqrt(2. / hn_3)))
    b_4 = tf.Variable(tf.zeros([num_labels]))
    lr = tf.Variable(0.1)
    
    def ForwardProp(dataset):
        z1 = tf.nn.relu(tf.matmul(dataset, w_1) + b_1)
        z2 = tf.nn.relu(tf.matmul(z1, w_2) + b_2)
        z3 = tf.nn.relu(tf.matmul(z2, w_3) + b_3)
        z4 = tf.matmul(z3, w_4) + b_4
        return z4

    ''' Loss - log the loss and see its behavior and change learning rate (try linear) '''
    ''' batch size - try to play with. '''
    # Training computation.
    logits = ForwardProp(tf_train_dataset)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) \
        + beta_val * ( tf.nn.l2_loss(w_1) + tf.nn.l2_loss(w_2) + tf.nn.l2_loss(w_3) + tf.nn.l2_loss(w_4)) \
        + beta_val * ( tf.nn.l2_loss(b_1) + tf.nn.l2_loss(b_2) + tf.nn.l2_loss(b_3) + tf.nn.l2_loss(b_4))

    # Optimizer.
    global_step = tf.Variable(0)  # count the number of steps taken.
    
    learning_rate = tf.train.polynomial_decay(0.9, global_step, num_steps - 1)

    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax( ForwardProp(tf_valid_dataset) )
    test_prediction = tf.nn.softmax( ForwardProp(tf_test_dataset) )

In [21]:
def runSession(beta):
    with tf.Session(graph=graph) as session:
      tf.global_variables_initializer().run()
      print("Initialized")
      for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, beta_val : beta}
        # the return value is the same as the first argument (named "parameters")]
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 250 == 0):
          print("Minibatch loss at step %d: %f" % (step, l))
          print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
          print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
      print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))
beta_vals = np.random.choice(np.logspace(-4,-2,20), 1) # [0.001] #np.logspace(-4, -2, 3)
for b in beta_vals:
    print('-----------')
    print('beta:',b)
    runSession(b)

-----------
beta: 0.0003359818286283781
Initialized
Minibatch loss at step 0: 3.340083
Minibatch accuracy: 10.9%
Validation accuracy: 23.6%
Minibatch loss at step 250: 1.363195
Minibatch accuracy: 86.7%
Validation accuracy: 84.4%
Minibatch loss at step 500: 1.123221
Minibatch accuracy: 89.8%
Validation accuracy: 86.1%
Minibatch loss at step 750: 1.155248
Minibatch accuracy: 85.9%
Validation accuracy: 86.9%
Minibatch loss at step 1000: 1.082448
Minibatch accuracy: 85.2%
Validation accuracy: 87.2%
Minibatch loss at step 1250: 0.924326
Minibatch accuracy: 88.3%
Validation accuracy: 87.2%
Minibatch loss at step 1500: 0.763982
Minibatch accuracy: 92.2%
Validation accuracy: 88.0%
Minibatch loss at step 1750: 0.756055
Minibatch accuracy: 92.2%
Validation accuracy: 88.2%
Minibatch loss at step 2000: 0.661748
Minibatch accuracy: 93.0%
Validation accuracy: 88.6%
Minibatch loss at step 2250: 0.851340
Minibatch accuracy: 83.6%
Validation accuracy: 87.4%
Minibatch loss at step 2500: 0.645449
Miniba

# In the following layer:
1. added dropout
2. l2-regularization - reduced
3. smaller layers
4. larger batch size
 -> for 6000 steps reached 96.1%.
 -> for 12000 steps reached 96.5

In [35]:
''' Multilayer '''

num_steps = 12001
batch_size = 128*2
hn_1 = 1024
hn_2 = 300
hn_3 = 50
image_size = 28
num_labels = 10

graph = tf.Graph()
with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    beta_val = tf.placeholder(tf.float32)
    
    # Variables.
    w_1 = tf.Variable(
        tf.truncated_normal( [image_size * image_size, hn_1], stddev=np.sqrt(2.0 / (image_size * image_size) )))
    b_1 = tf.Variable(tf.zeros([hn_1]))
    w_2 = tf.Variable(tf.truncated_normal([hn_1, hn_2], stddev=np.sqrt(2. / hn_1)))
    b_2 = tf.Variable(tf.zeros([hn_2]))
    w_3 = tf.Variable(tf.truncated_normal([hn_2, hn_3], stddev=np.sqrt(2. / hn_2)))
    b_3 = tf.Variable(tf.zeros([hn_3]))
    w_4 = tf.Variable(tf.truncated_normal([hn_3, num_labels], stddev=np.sqrt(2. / hn_3)))
    b_4 = tf.Variable(tf.zeros([num_labels]))
    lr = tf.Variable(0.1)
    
    def ForwardProp(dataset, dropout_prob=1.):
        z1 = tf.nn.relu(tf.nn.dropout(tf.matmul(dataset, w_1) + b_1, dropout_prob))
        z2 = tf.nn.relu(tf.matmul(z1, w_2) + b_2)
        z3 = tf.nn.relu(tf.matmul(z2, w_3) + b_3)
        z4 = tf.matmul(z3, w_4) + b_4
        return z4

    ''' Loss - log the loss and see its behavior and change learning rate (try linear) '''
    ''' batch size - try to play with. '''
    # Training computation.
    logits = ForwardProp(tf_train_dataset, 0.5)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) \
        + beta_val * ( tf.nn.l2_loss(w_1) + tf.nn.l2_loss(w_2) + tf.nn.l2_loss(w_3) + tf.nn.l2_loss(w_4)) \
        + beta_val * ( tf.nn.l2_loss(b_1) + tf.nn.l2_loss(b_2) + tf.nn.l2_loss(b_3) + tf.nn.l2_loss(b_4))

    # Optimizer.
    global_step = tf.Variable(0)  # count the number of steps taken.
    
    learning_rate = tf.train.polynomial_decay(0.99, global_step, num_steps - 1)

    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax( ForwardProp(tf_valid_dataset) )
    test_prediction = tf.nn.softmax( ForwardProp(tf_test_dataset) )

In [36]:
def runSession(beta):
    with tf.Session(graph=graph) as session:
      tf.global_variables_initializer().run()
      print("Initialized")
      for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, beta_val : beta}
        # the return value is the same as the first argument (named "parameters")]
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 250 == 0):
          print("Minibatch loss at step %d: %f" % (step, l))
          print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
          print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
        if (step % 1000 == 0):
            print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))
      print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))
    
beta_vals = np.random.choice(np.logspace(-4,-2,20), 1) * 0.01 # [0.001] #np.logspace(-4, -2, 3)
for b in beta_vals:
    print('-----------')
    print('beta:',b)
    runSession(b)

-----------
beta: 1.623776739188721e-06
Initialized
Minibatch loss at step 0: 2.403469
Minibatch accuracy: 7.8%
Validation accuracy: 29.4%
Test accuracy: 31.9%
Minibatch loss at step 250: 0.483596
Minibatch accuracy: 85.2%
Validation accuracy: 85.2%
Minibatch loss at step 500: 0.562236
Minibatch accuracy: 81.6%
Validation accuracy: 85.1%
Minibatch loss at step 750: 0.404082
Minibatch accuracy: 87.5%
Validation accuracy: 86.4%
Minibatch loss at step 1000: 0.395162
Minibatch accuracy: 87.5%
Validation accuracy: 87.4%
Test accuracy: 93.7%
Minibatch loss at step 1250: 0.417356
Minibatch accuracy: 85.5%
Validation accuracy: 87.6%
Minibatch loss at step 1500: 0.473535
Minibatch accuracy: 85.9%
Validation accuracy: 88.2%
Minibatch loss at step 1750: 0.370052
Minibatch accuracy: 88.3%
Validation accuracy: 88.3%
Minibatch loss at step 2000: 0.394140
Minibatch accuracy: 90.2%
Validation accuracy: 88.3%
Test accuracy: 94.4%
Minibatch loss at step 2250: 0.494521
Minibatch accuracy: 84.4%
Validatio

# Trying different power (using polynomial learning rate)
-> same code resulted at 96.4%

In [39]:
''' Multilayer '''

num_steps = 12001
batch_size = 128*2
hn_1 = 1024
hn_2 = 300
hn_3 = 50
image_size = 28
num_labels = 10

graph = tf.Graph()
with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    beta_val = tf.placeholder(tf.float32)
    
    # Variables.
    w_1 = tf.Variable(
        tf.truncated_normal( [image_size * image_size, hn_1], stddev=np.sqrt(2.0 / (image_size * image_size) )))
    b_1 = tf.Variable(tf.zeros([hn_1]))
    w_2 = tf.Variable(tf.truncated_normal([hn_1, hn_2], stddev=np.sqrt(2. / hn_1)))
    b_2 = tf.Variable(tf.zeros([hn_2]))
    w_3 = tf.Variable(tf.truncated_normal([hn_2, hn_3], stddev=np.sqrt(2. / hn_2)))
    b_3 = tf.Variable(tf.zeros([hn_3]))
    w_4 = tf.Variable(tf.truncated_normal([hn_3, num_labels], stddev=np.sqrt(2. / hn_3)))
    b_4 = tf.Variable(tf.zeros([num_labels]))
    lr = tf.Variable(0.1)
    global_step = tf.Variable(0)  # count the number of steps taken.
    
    if (global_step == 0):
        print(w_1)
        print(w_2)
        print(w_3)
        print(w_4)
    
    def ForwardProp(dataset, dropout_prob=1.):
        z1 = tf.nn.relu(tf.nn.dropout(tf.matmul(dataset, w_1) + b_1, dropout_prob))
        z2 = tf.nn.relu(tf.matmul(z1, w_2) + b_2)
        z3 = tf.nn.relu(tf.matmul(z2, w_3) + b_3)
        z4 = tf.matmul(z3, w_4) + b_4
        return z4

    ''' Loss - log the loss and see its behavior and change learning rate (try linear) '''
    ''' batch size - try to play with. '''
    # Training computation.
    logits = ForwardProp(tf_train_dataset, 0.5)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) \
        + beta_val * ( tf.nn.l2_loss(w_1) + tf.nn.l2_loss(w_2) + tf.nn.l2_loss(w_3) + tf.nn.l2_loss(w_4)) \
        + beta_val * ( tf.nn.l2_loss(b_1) + tf.nn.l2_loss(b_2) + tf.nn.l2_loss(b_3) + tf.nn.l2_loss(b_4))

    # Optimizer.    
    learning_rate = tf.train.polynomial_decay(0.99, global_step, num_steps - 1, power=0.5)

    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax( ForwardProp(tf_valid_dataset) )
    test_prediction = tf.nn.softmax( ForwardProp(tf_test_dataset) )

In [38]:
def runSession(beta):
    with tf.Session(graph=graph) as session:
      tf.global_variables_initializer().run()
      print("Initialized")
      for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, beta_val : beta}
        # the return value is the same as the first argument (named "parameters")]
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 250 == 0):
          print("Minibatch loss at step %d: %f" % (step, l))
          print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
          print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
        if (step % 1000 == 0):
            print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))
      print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))
    
beta_vals = np.random.choice(np.logspace(-4,-2,20), 1) * 0.01 # [0.001] #np.logspace(-4, -2, 3)
for b in beta_vals:
    print('-----------')
    print('beta:',b)
    runSession(b)

-----------
beta: 1.623776739188721e-06
Initialized
Minibatch loss at step 0: 2.388157
Minibatch accuracy: 12.1%
Validation accuracy: 22.0%
Test accuracy: 23.0%
Minibatch loss at step 250: 0.495060
Minibatch accuracy: 83.6%
Validation accuracy: 84.5%
Minibatch loss at step 500: 0.553063
Minibatch accuracy: 84.8%
Validation accuracy: 83.1%
Minibatch loss at step 750: 0.380174
Minibatch accuracy: 88.3%
Validation accuracy: 86.7%
Minibatch loss at step 1000: 0.400405
Minibatch accuracy: 88.3%
Validation accuracy: 86.8%
Test accuracy: 93.1%
Minibatch loss at step 1250: 0.414342
Minibatch accuracy: 85.9%
Validation accuracy: 87.5%
Minibatch loss at step 1500: 0.504027
Minibatch accuracy: 86.3%
Validation accuracy: 88.0%
Minibatch loss at step 1750: 0.334358
Minibatch accuracy: 89.5%
Validation accuracy: 88.2%
Minibatch loss at step 2000: 0.374067
Minibatch accuracy: 87.1%
Validation accuracy: 88.0%
Test accuracy: 94.0%
Minibatch loss at step 2250: 0.482542
Minibatch accuracy: 83.6%
Validati

# Now - with larger batch side
Yoooohoooo 96.9%

In [41]:
''' Multilayer '''

num_steps = 12001
batch_size = 512
hn_1 = 1024
hn_2 = 300
hn_3 = 50
image_size = 28
num_labels = 10

graph = tf.Graph()
with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    beta_val = tf.placeholder(tf.float32)
    
    # Variables.
    w_1 = tf.Variable(
        tf.truncated_normal( [image_size * image_size, hn_1], stddev=np.sqrt(2.0 / (image_size * image_size) )))
    b_1 = tf.Variable(tf.zeros([hn_1]))
    w_2 = tf.Variable(tf.truncated_normal([hn_1, hn_2], stddev=np.sqrt(2. / hn_1)))
    b_2 = tf.Variable(tf.zeros([hn_2]))
    w_3 = tf.Variable(tf.truncated_normal([hn_2, hn_3], stddev=np.sqrt(2. / hn_2)))
    b_3 = tf.Variable(tf.zeros([hn_3]))
    w_4 = tf.Variable(tf.truncated_normal([hn_3, num_labels], stddev=np.sqrt(2. / hn_3)))
    b_4 = tf.Variable(tf.zeros([num_labels]))
    lr = tf.Variable(0.1)
    global_step = tf.Variable(0)  # count the number of steps taken.
    
    if (global_step == 0):
        print(w_1)
        print(w_2)
        print(w_3)
        print(w_4)
    
    def ForwardProp(dataset, dropout_prob=1.):
        z1 = tf.nn.relu(tf.nn.dropout(tf.matmul(dataset, w_1) + b_1, dropout_prob))
        z2 = tf.nn.relu(tf.matmul(z1, w_2) + b_2)
        z3 = tf.nn.relu(tf.matmul(z2, w_3) + b_3)
        z4 = tf.matmul(z3, w_4) + b_4
        return z4

    ''' Loss - log the loss and see its behavior and change learning rate (try linear) '''
    ''' batch size - try to play with. '''
    # Training computation.
    logits = ForwardProp(tf_train_dataset, 0.5)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) \
        + beta_val * ( tf.nn.l2_loss(w_1) + tf.nn.l2_loss(w_2) + tf.nn.l2_loss(w_3) + tf.nn.l2_loss(w_4)) \
        + beta_val * ( tf.nn.l2_loss(b_1) + tf.nn.l2_loss(b_2) + tf.nn.l2_loss(b_3) + tf.nn.l2_loss(b_4))

    # Optimizer.    
    learning_rate = tf.train.polynomial_decay(0.99, global_step, num_steps - 1, power=0.5)

    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax( ForwardProp(tf_valid_dataset) )
    test_prediction = tf.nn.softmax( ForwardProp(tf_test_dataset) )

In [42]:
def runSession(beta):
    with tf.Session(graph=graph) as session:
      tf.global_variables_initializer().run()
      print("Initialized")
      for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, beta_val : beta}
        # the return value is the same as the first argument (named "parameters")]
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 250 == 0):
          print("Minibatch loss at step %d: %f" % (step, l))
          print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
          print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
        if (step % 1000 == 0):
            print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))
      print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))
    
beta_vals = np.random.choice(np.logspace(-4,-2,20), 1) * 0.01 # [0.001] #np.logspace(-4, -2, 3)
for b in beta_vals:
    print('-----------')
    print('beta:',b)
    runSession(b)

-----------
beta: 4.832930238571752e-05
Initialized
Minibatch loss at step 0: 2.444844
Minibatch accuracy: 11.1%
Validation accuracy: 10.3%
Test accuracy: 10.3%
Minibatch loss at step 250: 0.558319
Minibatch accuracy: 85.2%
Validation accuracy: 85.4%
Minibatch loss at step 500: 0.535825
Minibatch accuracy: 84.8%
Validation accuracy: 87.1%
Minibatch loss at step 750: 0.442988
Minibatch accuracy: 87.1%
Validation accuracy: 87.7%
Minibatch loss at step 1000: 0.438248
Minibatch accuracy: 87.1%
Validation accuracy: 88.1%
Test accuracy: 94.6%
Minibatch loss at step 1250: 0.378720
Minibatch accuracy: 89.8%
Validation accuracy: 88.7%
Minibatch loss at step 1500: 0.405607
Minibatch accuracy: 89.6%
Validation accuracy: 89.0%
Minibatch loss at step 1750: 0.467977
Minibatch accuracy: 88.1%
Validation accuracy: 88.8%
Minibatch loss at step 2000: 0.458918
Minibatch accuracy: 88.5%
Validation accuracy: 89.4%
Test accuracy: 95.2%
Minibatch loss at step 2250: 0.422247
Minibatch accuracy: 87.9%
Validati

# Higher batch size
as the losses were not so good on the last try, not decaying like I wanted to
batch size of 1024: 96.6%

In [43]:
''' Multilayer '''

num_steps = 12001
batch_size = 1024
hn_1 = 1024
hn_2 = 300
hn_3 = 50
image_size = 28
num_labels = 10

graph = tf.Graph()
with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    beta_val = tf.placeholder(tf.float32)
    
    # Variables.
    w_1 = tf.Variable(
        tf.truncated_normal( [image_size * image_size, hn_1], stddev=np.sqrt(2.0 / (image_size * image_size) )))
    b_1 = tf.Variable(tf.zeros([hn_1]))
    w_2 = tf.Variable(tf.truncated_normal([hn_1, hn_2], stddev=np.sqrt(2. / hn_1)))
    b_2 = tf.Variable(tf.zeros([hn_2]))
    w_3 = tf.Variable(tf.truncated_normal([hn_2, hn_3], stddev=np.sqrt(2. / hn_2)))
    b_3 = tf.Variable(tf.zeros([hn_3]))
    w_4 = tf.Variable(tf.truncated_normal([hn_3, num_labels], stddev=np.sqrt(2. / hn_3)))
    b_4 = tf.Variable(tf.zeros([num_labels]))
    lr = tf.Variable(0.1)
    global_step = tf.Variable(0)  # count the number of steps taken.
    
    if (global_step == 1):
        print(w_1)
        print(w_2)
        print(w_3)
        print(w_4)
    
    def ForwardProp(dataset, dropout_prob=1.):
        z1 = tf.nn.relu(tf.nn.dropout(tf.matmul(dataset, w_1) + b_1, dropout_prob))
        z2 = tf.nn.relu(tf.matmul(z1, w_2) + b_2)
        z3 = tf.nn.relu(tf.matmul(z2, w_3) + b_3)
        z4 = tf.matmul(z3, w_4) + b_4
        return z4

    ''' Loss - log the loss and see its behavior and change learning rate (try linear) '''
    ''' batch size - try to play with. '''
    # Training computation.
    logits = ForwardProp(tf_train_dataset, 0.5)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) \
        + beta_val * ( tf.nn.l2_loss(w_1) + tf.nn.l2_loss(w_2) + tf.nn.l2_loss(w_3) + tf.nn.l2_loss(w_4)) \
        + beta_val * ( tf.nn.l2_loss(b_1) + tf.nn.l2_loss(b_2) + tf.nn.l2_loss(b_3) + tf.nn.l2_loss(b_4))

    # Optimizer.    
    learning_rate = tf.train.polynomial_decay(0.99, global_step, num_steps - 1, power=0.5)

    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax( ForwardProp(tf_valid_dataset) )
    test_prediction = tf.nn.softmax( ForwardProp(tf_test_dataset) )

In [45]:
def runSession(beta):
    with tf.Session(graph=graph) as session:
      tf.global_variables_initializer().run()
      print("Initialized")
      for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, beta_val : beta}
        # the return value is the same as the first argument (named "parameters")]
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 250 == 0):
          print("Minibatch loss at step %d: %f" % (step, l))
          print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
          print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
        if (step % 1000 == 0):
            print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))
      print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))
    
beta_vals = [0.0001] #np.random.choice(np.logspace(-4,-2,20), 1) * 0.01 # [0.001] #np.logspace(-4, -2, 3)
for b in beta_vals:
    print('-----------')
    print('beta:',b)
    runSession(b)

-----------
beta: 0.0001
Initialized
Minibatch loss at step 0: 2.556504
Minibatch accuracy: 8.9%
Validation accuracy: 31.9%
Test accuracy: 35.5%
Minibatch loss at step 250: 0.609848
Minibatch accuracy: 85.5%
Validation accuracy: 86.2%
Minibatch loss at step 500: 0.551132
Minibatch accuracy: 86.0%
Validation accuracy: 87.8%
Minibatch loss at step 750: 0.492023
Minibatch accuracy: 88.1%
Validation accuracy: 88.5%
Minibatch loss at step 1000: 0.426389
Minibatch accuracy: 90.5%
Validation accuracy: 89.1%
Test accuracy: 94.7%
Minibatch loss at step 1250: 0.431014
Minibatch accuracy: 89.6%
Validation accuracy: 89.4%
Minibatch loss at step 1500: 0.419734
Minibatch accuracy: 89.2%
Validation accuracy: 89.6%
Minibatch loss at step 1750: 0.388627
Minibatch accuracy: 90.5%
Validation accuracy: 89.8%
Minibatch loss at step 2000: 0.458749
Minibatch accuracy: 89.0%
Validation accuracy: 90.0%
Test accuracy: 95.6%
Minibatch loss at step 2250: 0.396520
Minibatch accuracy: 90.5%
Validation accuracy: 90.